# *Important!*

Before running this notebook, make sure that you:

1. Have a designated folder/directory that you can access and set as a current working directory.
2. Cloned the [`halphagui` repository](https://github.com/rfinn/halphagui) to somewhere accessible on your device.
3. Created a python virtual environment with python version 3.9 and `requirements.txt` installed.
4. Created a python kernel for the environment that can be accessed in Jupyter Lab. 
    - [This link](https://github.com/rfinn/halphagui/wiki/1---Installation) leads you to the instructions for the setup of a python virtual environment.
5. Have `swarp` installed on your device.
    - [This link](https://www.macports.org/install.php) leads you to the steps for installing `swarp` through MacPorts.
    - After installing MacPorts, you can install `swarp` by running this line in your terminal:
        - `port install swarp`

Make sure you go to [this link](https://github.com/rfinn/halphagui) to clone the repository and set up a virtual or conda environment, or else you will not be able to run this notebook! Once you set up the environment and the kernel, you can come back and start the tutorial.

The instructions for setting up the environment are at the bottom of the repo, or you can just go to [this link](https://github.com/rfinn/halphagui/wiki/1---Installation). I recommend setuping up a conda environment, but the choice is yours.

# Imports

In [ ]:
# No need to change or add anything to this cell, all you need to do is run it!

import os
import sys
import numpy as np
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
from astropy.table import Table,Column
from astropy.io import fits
from PIL import Image
import warnings
import glob
warnings.filterwarnings('ignore')

# Setting Home Directory and CWD

In [ ]:
# Let's first get our home environment:

homedir = os.getenv('HOME')

# The current working directory (CWD) will be where all of the files will be generated and stored.
#
# Change the ... to what directory you want to work out of.
# formating example: '/Desktop/myGalaxyResearch/' 

cwd = homedir+'...'
os.chdir(cwd)

# We need to make a subfolder in this directory named 'data' where all of your
#tables will be stored.
#
# We also need to make a subfolder 'galPlots' for where the python plots will be stored.

if os.path.exists(cwd+'data/'):
    print('data subfolder already exists.')
else:
    os.mkdir(cwd+'data/')

# Now add the path to the halphagui repository to our system.
# Make sure you clone the repository first and have it stored somewhere on your computer.
#
# Change the ... to the path that points to where the cloned repository is.
# formatting example: '/clonedRepos/halphagui/'

sys.path.append(homedir+'...')

# These imports cannot be part of the first round of imports because these are in the halphagui repo.

import photwrapper
from image_functions import *

# Reading in your Table

In [ ]:
# Now read in your table of galaxies that will be used to generate stellar masses.
# This table MUST HAVE ~at least~ these specific column headers:
#
# 'RA' , 'DEC' , 'AGCnr' , 'mu' , and 'e_mu'
#
# 'tablepath' will be the path that points to where the table you want to use is!

tablepath = '...'
    
myGalTab=Table.read(tablepath)

# Functions for calculations

In [ ]:
# run me!

#################################################################################

def getLW1(Fenc,mu,e_mu,m):
    '''
    Takes in ptab and matched table values for distance modulus plus error, then calculates the luminosity two
    different ways; one in relation to the distance modulus and absolute magnitude, and the other through the flux-
    luminosity relation using the max enclosed flux from photutils.

    Inputs:
        Fenc = max enclosed flux of target galaxy; ergs/cm^2 s
        mu = target galaxy distance modulus
        e_mu = target galaxy error in the distance modulus
        m = source apparent magnitude of target galaxy

    RETURNS:
        LW1_mag = numpy array including the upper, median, and lower W1 Luminosities via distance modulus, mu, 
                     and absolute magnitude of galaxy; erg/s.
        LW1_flux = numpy array including the upper, median, and lower W1 Luminosities via flux-luminosity relation;
                     erg/s.
    '''
    
    cmConversion = 3.086e18
    d_Upper=(10**((mu+e_mu)/5+1))*cmConversion # gets d in pc, converts to cm; includes upper error for mu
    d_Med=(10**(mu/5+1))*cmConversion # gets d in pc, converts to cm
    d_Lower=(10**((mu-e_mu)/5+1))*cmConversion # gets d in pc, converts to cm; includes lower error for mu
    d=np.array([d_Upper,d_Med,d_Lower])

    M_Upper=m-(mu+e_mu)
    M_Med=m-mu
    M_Lower=m-(mu-e_mu)
    
    M=np.array([M_Upper,M_Med,M_Lower])
    
    LW1_mag_Upper=10**(-0.4*(M_Upper-3.24))
    LW1_mag_Med=10**(-0.4*(M_Med-3.24))
    LW1_mag_Lower=10**(-0.4*(M_Lower-3.24))
    
    LW1_mag=np.array([LW1_mag_Upper,LW1_mag_Med,LW1_mag_Lower])

    ### Luminosity via enclosed flux
    Lsun_ergs = 3.846e33
    LW1_flux_Upper=Fenc*(4*np.pi*d_Upper**2)/Lsun_ergs
    LW1_flux_Med=Fenc*(4*np.pi*d_Med**2)/Lsun_ergs
    LW1_flux_Lower=Fenc*(4*np.pi*d_Lower**2)/Lsun_ergs
    
    LW1_flux=np.array([LW1_flux_Upper,LW1_flux_Med,LW1_flux_Lower])

    return LW1_mag,LW1_flux

#################################################################################

def getPlots(ptab,galID,image_names):
    '''
    Generates plots for the enclosed flux and source magnitude versus semi-major axis from the
    photometry table via photwrapper. These plots are saved under the galPlots subfolder.
    
    Inputs:
        ptab = photwrapper output table for a given galaxy containing data for flux and magnitude for
                for intervals of semi-major axis.
        galID = used only for the title of the plot, has both the AGC prefix and identification number
                 out to 6 digits.
        image_names = list of the image file names for WISE W1-4.
    '''
    
    SMA = ptab['sma_arcsec'] # semi-major axis measured in arcseconds
    FLUX = ptab['flux_erg'] # enclosed flux measured in ergs
    MAG = ptab['mag'] # source magnitude through each fitted ellipse
    
    plt.figure(figsize=(12,5))
    
    plt.subplot(1,2,1)
    plt.plot(SMA,FLUX)
    plt.title(f'{galID} $F_{{enc}}$ Vs. SMA',fontsize=16)
    plt.xlabel("Semi-major axis (arcsec)",fontsize=14)
    plt.ylabel("$F_{enc}$ (erg / $cm^2$ s)",fontsize=14)
    
    plt.subplot(1,2,2)
    plt.plot(SMA,MAG)
    plt.title(f'{galID} Magnitude Vs. SMA',fontsize=16)
    plt.xlabel("Semi-major axis (arcsec)",fontsize=14)
    plt.ylabel("Magnitude",fontsize=14)    
    
    ptabimpath = cwd+f'{galID}/figures/'+f'{galID}-ptab-plots.png'
    if os.path.exists(ptabimpath):
        os.remove(ptabimpath)
    plt.savefig(ptabimpath)
    plt.close()

    plt.figure(figsize=(12,6.5))
    # plot WISE images
    image_names.sort()
    imname = image_names[0]
    imnames = ['W1','W2','W3','W4']
    for i,im in enumerate(image_names):
        plt.subplot(1,4,i+1)
        data = fits.getdata(im)
        display_image(data,percent=92)
        plt.title(imnames[i],fontsize=14)
            
    hdu = fits.open(imname)
    data = hdu[0].data
    hdu.close()
    
    W1impath = cwd+f'{galID}/figures/'+f'{galID}-W1cutout.png'
    if os.path.exists(W1impath):
        os.remove(W1impath)
    plt.savefig(W1impath)
    plt.close()
    
#################################################################################

def getPhot(imname,galID):
    '''
    Uses photwrapper, the halphagui repo photometry calculator for a detected galaxy through various 
    photutils methods/functions, to fit concentric ellipses on both the detected galaxy and the background sources.
    The background sources are masked out and the flux is measured at each ring; based on SMA.
    
    Inputs:
        imname = the selected image that you want to measure the flux of. For this these stellar mass calculations,
                     the W1.fits image for each galaxy will be used for the calculations, however you can input other files.
        galID = AGC prefix and identification number for target galaxy out to 6 digits.

    Returns:
        ptab = photwrapper output table for a given galaxy containing data for flux and magnitude for
                for intervals of semi-major axis. 
        e = photwrapper ellipse fitting/analysis for target galaxy.
    '''
        
    # check if mask exists
    maskname = imname.replace('.fits','-mask.fits')
    maskpath = cwd+f'{galID}/imdat/{maskname}'
    
    # looking for mask_wrapper file
    if os.path.exists(maskpath):
        maskfile = maskpath
        print(f'\nMaskfile for {galID} found! No need to generate one.\n')

        e = photwrapper.ellipse(imname,mask=maskpath)

    else:
        print(f'\nNo maskfile for {galID} found! Using maskwrapper.py to generate one now.\n')
        
        # call the mask wrapper from within your script
        os.system(f"python ~/github/halphagui/maskwrapper.py --image {imname} --ngrow {1} --sesnr {2} --minarea {5} --auto")
        maskfile = maskpath
    
        e = photwrapper.ellipse(imname,mask=maskname)
    
    e.detect_objects()
    e.find_central_object()    
    e.get_ellipse_guess()    
    e.measure_phot()     
    e.calc_sb()
    e.convert_units()
    e.write_phot_tables()
    e.write_phot_fits_tables()
    
    allApertures_filename = f'{galID}-allApertures.png'
    e.show_seg_aperture(plotname=allApertures_filename)
    plt.savefig(cwd+f'{galID}/figures/{allApertures_filename}')
    plt.close()
    
    photApertures_filename = f'{galID}-photApertures.png'
    e.draw_phot_apertures(plotname=photApertures_filename)
    plt.savefig(cwd+f'{galID}/figures/{photApertures_filename}')
    plt.close()
    
    ptabName = imname.replace('.fits','_phot.fits')
    ptab = Table.read(ptabName)
    
    return ptab,e

#################################################################################

def getLogMass(LW1_array):
    '''
    Reads in the W1 luminosities calculations with the +/- error on the distance modulus, including the 
    calculation without error propagation, and converts those luminosities into stellar masses, via 
    equation from Jarrett 2023.

    Inputs:
        LW1_array = contains the upper, median, and lower luminosities as a (3,) shape array:
            LW1_upper = W1 luminosity including the +error on the distance modulus; distance modulus will 
                            be the largest source of error; erg/s
            LW1_med =  W1 luminosity NOT including the +/-error on the distance modulus; erg/s
            LW1_lower = W1 luminosity including the -error on the distance modulus; distance modulus will 
                            be the largest source of error; erg/s
        
    RETURNS:
        logMstar = numpy array of including the upper, median, and lower W1 stellar mass calculations.
    '''
    
    A0= -12.62185; A1= 5.00155; A2= -0.43857; A3= 0.01593
    
    logMstar_upper=A0+(A1*np.log10(LW1_array[0]))+(A2*(np.log10(LW1_array[0]))**2)+(A3*(np.log10(LW1_array[0]))**3)
    logMstar_med=A0+(A1*np.log10(LW1_array[1]))+(A2*(np.log10(LW1_array[1]))**2)+(A3*(np.log10(LW1_array[1]))**3)
    logMstar_lower=A0+(A1*np.log10(LW1_array[2]))+(A2*(np.log10(LW1_array[2]))**2)+(A3*(np.log10(LW1_array[2]))**3)
    
    logMstar=np.array([logMstar_upper,logMstar_med,logMstar_lower])
    
    return logMstar

#################################################################################

# Main function that pulls it all together

In [ ]:
def getMasses(galTab,verbose=False):
    '''
    Reads in a table of galaxy coordinates to get wise/legacy images to calculate photometry; W1 phot will be converted
    into a flux, then converted into a stellar mass.
    
    Inputs:
        galTab = table that has galaxies that need to have stellar masses calculated, was read in at the beginning of the notebook.
                    Must have these columns: 'RA' , 'DEC' , 'AGCnr' , 'Imsize' , 'mu' , and 'e_mu' !!!
        verbose = conditional to have function talk to you through the processes within; be verbose!
    
    RETURNS:
        galTable_withMasses = duplicate table of galTab but includes new columns for the calculated stellar masses
                                for magnitude and flux, plus their upper and lower calculations, too.
    '''
    
    for row in galTab:
        
        # get sky coords and galaxy AGC ID from input table
        ind = row.index
        ra = float(row['RA'])
        dec = float(row['DEC'])
        galID = f"AGC{row['AGCnr']:06d}"; galNum = row['AGCnr']
        imgsize = 120

        # creates data folder for individual galaxy, then go into it
        if not os.path.exists(cwd+galID):
            os.mkdir(cwd+galID)

        # looks to see if the image data subdirectories need to be made.
        if not os.path.exists(cwd+f'{galID}/imdat/'):
            os.mkdir(cwd+f'{galID}/imdat/')
        
        # looks to see if the plotting subdirectories need to be made.
        if not os.path.exists(cwd+f'{galID}/figures/'):
            os.mkdir(cwd+f'{galID}/figures/')
        
        # setting pixel scaling
        UNWISE_PIXSCALE = 2.75
        LEGACY_PIXSCALE = 1

        ###########################
        ### Getting WISE Images ###
        ###########################
        
        try:
            image_names,weight_names,multiframe = get_unwise_image(ra, dec, galid=galID,
                                                    pixscale=UNWISE_PIXSCALE, imsize=imgsize, bands='1234',
                                                    makeplots=False,subfolder=None,verbose=False)
        except:
            print(f'\nWarning: Problem with getting UNWISE images for {galID}!\n')
            continue

        # We need to sort the images because sometimes the images will be out of order in the array.
        # This ensures that we always grab the W1 image for the calculations, but it can be changed if needed.

        image_names.sort()
        imname = image_names[0]

        ##########################################
        ### Maskwrapper & Photwrapper Analysis ###
        ##########################################
        ptab,e=getPhot(imname,galID)

        try:
            ptab,e=getPhot(imname,galID)
        except:
            print(f'\nWarning: Problem with photometry for {galID}!\n')
            continue

        # Plot F_enc vs SMA and Mag vs SMA, then save the plot as .png in galPlots folder.
        getPlots(ptab,galID,image_names)
        
        #################################
        ### Stellar Mass Calculations ###
        #################################
    
        maxFenc=np.max(ptab['flux_erg']) # max enclosed flux in ergs/cm^2 s
        mu=row['mu'] # distance modulus
        mu_err=row['e_mu'] # distance modulus error
        mmag=np.min(ptab['mag']) # source magnitude via photutils

        LW1_mag,LW1_flux=getLW1(maxFenc,mu,mu_err,mmag)        

        logMstar_mag=getLogMass(LW1_mag).round(5)
        logMstar_flux=getLogMass(LW1_flux).round(5)

        massPath = cwd+'data/galTable_withMasses.fits'
        
        if os.path.isfile(massPath):
            # reads in the masses table
            massTab = Table.read(massPath)
            
        else:
            # creates a duplicate table
            massTab = Table.read(tablepath)          

            # creates columns for stellar masses
            flux_col_Median = Column(0.0, name='logMstar_flux_Median'); massTab.add_column(flux_col_Median)
            flux_col_Upper = Column(0.0, name='logMstar_flux_Upper'); massTab.add_column(flux_col_Upper)
            flux_col_Lower = Column(0.0, name='logMstar_flux_Lower'); massTab.add_column(flux_col_Lower)
            
        massTab['logMstar_flux_Median'][ind]=logMstar_flux[1]
        massTab['logMstar_flux_Upper'][ind]=logMstar_flux[0]
        massTab['logMstar_flux_Lower'][ind]=logMstar_flux[2]

        # this rewrites the galTable_withMasses.fits file in the data folder every time a new mass is calculated,
        # which will keep track of all the masses that have been done without altering the primary fits file!
        massTab.write(massPath,format='fits',overwrite=True)

        # gathering all of the plots made and removing/renaming them to fall
        # into respective galaxy image folder
        galplots=glob.glob(galID+'*.png')
        for img in galplots:
            os.remove(img)
        
        imdata=glob.glob(galID+'-*')
        for img in imdata:
            if not os.path.exists(cwd+f'{galID}/imdat/{img}'):
                os.rename(img,cwd+f'{galID}/imdat/{img}')
            else:
                os.remove(img)
    
    return massTab

# Calling the functions

In [ ]:
# We can run the entire table to get the stellar masses using the getMasses function!
# Make sure that the table that you called in has the appropriate columns!!
myMassTab = getMasses(
    
    # Instead of just running one row at a time, we can just put the whole table in!
    # This is the same table that was read in from the beginning of the tutorial.
    myGalTab
    
)